In [ ]:
%matplotlib inline
# %matplotlib

In [ ]:
# Loading libaries

import datetime
import os
import pickle
import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors 
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('png')

plt.rcParams['savefig.dpi'] = 75
plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = (21, 30)  # A4
plt.rcParams['lines.linewidth'] = 1.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['text.usetex'] = False
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['xtick.major.pad']=12
matplotlib.rcParams['ytick.major.pad']=12
#matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

import numpy as np
import scipy as scipy
import scipy.stats as stats
import scipy.linalg as linalg
import xarray as xr
from pprint import pprint
import sys
#import os
import copy
import string
import glob
#import xarray as xr 

import warnings
#warnings.filterwarnings('ignore')

In [ ]:
Ibase_path = os.path.abspath('L63_FTLEs_DA_v2_penakalnay2004.ipynb')
#base_dir = '/g/data/v14/Projects/L63_dimension'

In [ ]:
now = datetime.datetime.now()
print('date : {0} / {1} / {2}'.format(now.day, now.month, now.year))
cwd = os.getcwd()
print('working directory : {0}'.format(cwd))
print('data directory : {0}'.format(cwd))

In [ ]:
from DA_module import *
from ftles import *
from CLVs import *

In [ ]:
cmaps = [('Perceptually Uniform Sequential', [
            'viridis', 'plasma', 'inferno', 'magma', 'cividis']),
         ('Sequential', [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']),
         ('Sequential (2)', [
            'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
            'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
            'hot', 'afmhot', 'gist_heat', 'copper']),
         ('Diverging', [
            'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
            'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic',
            'berlin', 'managua', 'vanimo']),
         ('Cyclic', ['twilight', 'twilight_shifted', 'hsv']),
         ('Qualitative', [
            'Pastel1', 'Pastel2', 'Paired', 'Accent',
            'Dark2', 'Set1', 'Set2', 'Set3',
            'tab10', 'tab20', 'tab20b', 'tab20c']),
         ('Miscellaneous', [
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
            'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
            'gist_ncar'])]

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))


def plot_color_gradients(cmap_category, cmap_list):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows-1)*0.1)*0.22
    fig, axs = plt.subplots(nrows=nrows, figsize=(6.4, figh))
    fig.subplots_adjust(top=1-.35/figh, bottom=.15/figh, left=0.2, right=0.99)

    axs[0].set_title(f"{cmap_category} colormaps", fontsize=14)

    for ax, cmap_name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=cmap_name)
        ax.text(-.01, .5, cmap_name, va='center', ha='right', fontsize=10,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()


#for cmap_category, cmap_list in cmaps:
 #   plot_color_gradients(cmap_category, cmap_list)
#plt.show()

# Control Run

In [ ]:
# Model properties
dt                = 0.005

parameters_spinup = 50     # skip this first number of analyses in determining statistics
forecast_window   = 25       # window between each DA cycle
dim  = 9
# control run
coupling = 'PenaKalnay2004'
num_steps = 100000
np.random.seed(seed=1)
IC     = np.ones(dim)
#IC_run = Lorenz63(num_time_steps=num_steps, dt=dt, IC=IC)
#IC_run = CoupledLorenz(num_time_steps=num_steps, dt=dt, coupling=coupling, IC=IC)
IC_run = PenaKalnay2004(num_time_steps=num_steps, dt=dt, IC=IC)  # IGNORE
IC_run.run()

In [ ]:
fig, a1 = plt.subplots(1,1,figsize=[20,10])
a1.plot(IC_run.time[:20000],IC_run.x[:,:20000].T)
a1.grid()
a1.set_xlabel('time')
plt.show()
 

In [ ]:
fig1 = plt.figure(figsize=[10,10])
a1 = fig1.add_subplot(1,1,1,projection='3d')
a1.plot(IC_run.x[0,4000:],IC_run.x[1,4000:],IC_run.x[2,4000:],'.',ms=2)
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
a1.set_zlabel('z')

plt.show()

In [ ]:
fig2 = plt.figure(figsize=[10,10])
a2 = fig2.add_subplot(1,1,1,projection='3d')
a2.plot(IC_run.x[3,4000:],IC_run.x[4,4000:],IC_run.x[5,4000:],'.',ms=2)
a2.grid()
a2.set_xlabel('X')
a2.set_ylabel('Y')
a2.set_zlabel('Z')
plt.show()

In [ ]:
fig3 = plt.figure(figsize=[10,10])
a2 = fig3.add_subplot(1,1,1,projection='3d')
a2.plot(IC_run.x[6,4000:],IC_run.x[7,4000:],IC_run.x[8,4000:],'.',ms=2)
a2.grid()
a2.set_xlabel('X')
a2.set_ylabel('Y')
a2.set_zlabel('Z')
plt.show()

# Dynamical Properties

In [ ]:
start_ind = 4000
start_time = dt*start_ind

start = time.time()
cocycle = IC_run.calculate_matrix_cocycle(IC_run.x,tlims=[start_time,dt*num_steps])
end = time.time()

comp_time = (end-start)

print('cocycle calculated in ' + str(np.round(comp_time,4)) + ' sec')

## FTLEs

In [ ]:
CLVS_DIR = os.path.join(cwd, 'CLVs')
print('dim', dim, 'shape=', cocycle.shape[2])
if not os.path.exists(CLVS_DIR):
    os.makedirs(CLVS_DIR)

Rdiag_filename = '.'.join(['PenaKalnay2004Rdiag','nc'])
Rdiag_file = os.path.join(CLVS_DIR, Rdiag_filename)

#if os.path.exists(Rdiag_file):
#    Rdiag = xr.open_dataset(Rdiag_file).Rdiag
#else:
start = time.time()
Rdiag = calculate_FTLEs(dim, cocycle, np.arange(0,cocycle.shape[2]))[1]
end = time.time()

comp_time = (end-start)/60

print('R matrices calculated in ' + str(np.round(comp_time,4)) + ' min')
    
    ## convert to xarray
Rdiag = xr.DataArray(Rdiag)
    
    # save to netCDF
Rdiag.to_dataset(name='PenaKalnay2004Rdiag').to_netcdf(Rdiag_file)

In [ ]:
window = 100

FTLEs = np.zeros((dim,num_steps-start_ind-window))

start = time.time()
for ti in np.arange(0,num_steps-window-start_ind):
    FTLEs[:,ti] = np.sum(np.log(Rdiag[:,ti:ti+window]),axis=1)/(window*dt)
end = time.time()
comp_time = (end-start)

print('calculated in ' + str(np.round(comp_time,4)) + ' sec')

In [ ]:
mycmap = matplotlib.colors.ListedColormap([
 [0., 0., 1., 1.      ],
 [0.25, 0.25, 1., 1.      ],
 [0.5, 0.5, 1., 1.      ], 
 [0.75, 0.75, 1., 1.      ],
 [1., 0.75, 0.75, 1.      ],
 [1., 0.5, 0.5, 1.      ],
 [1., 0.25, 0.25, 1.      ],
 [1., 0., 0., 1.      ]])

fig = plt.figure(figsize=[24,24],layout='constrained',dpi=300)
a1 = fig.add_subplot(3,3,1,projection='3d')
scat = a1.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[0,:],s=2,cmap='RdBu_r',vmax=7,vmin=-7)
a1.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a1.set_title('FTLE 1',fontsize=24)

a2 = fig.add_subplot(3,3,2,projection='3d')
scat = a2.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[1,:],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a2.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a2.set_title('FTLE 2',fontsize=24)

a3 = fig.add_subplot(3,3,3,projection='3d')
scat = a3.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=FTLEs[2,:],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a3.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a3.set_title('FTLE 3',fontsize=24)

a4 = fig.add_subplot(3,3,4,projection='3d')
scat = a4.scatter(IC_run.x[3,start_ind:-window],IC_run.x[4,start_ind:-window],IC_run.x[5,start_ind:-window],
           c=FTLEs[3,:],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a4.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a4.set_title('FTLE 4',fontsize=24)

a5 = fig.add_subplot(3,3,5,projection='3d')
scat = a5.scatter(IC_run.x[3,start_ind:-window],IC_run.x[4,start_ind:-window],IC_run.x[5,start_ind:-window],
           c=FTLEs[4,:],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a5.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a5.set_title('FTLE 5',fontsize=24)

a6 = fig.add_subplot(3,3,6,projection='3d')
scat = a6.scatter(IC_run.x[3,start_ind:-window],IC_run.x[4,start_ind:-window],IC_run.x[5,start_ind:-window],
           c=FTLEs[5,:],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a6.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a6.set_title('FTLE 6',fontsize=24)

a7 = fig.add_subplot(3,3,7,projection='3d')
scat = a7.scatter(IC_run.x[6,start_ind:-window],IC_run.x[7,start_ind:-window],IC_run.x[8,start_ind:-window],
           c=FTLEs[6,:],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a7.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a7.set_title('FTLE 7',fontsize=24)

a8 = fig.add_subplot(3,3,8,projection='3d')
scat = a8.scatter(IC_run.x[6,start_ind:-window],IC_run.x[7,start_ind:-window],IC_run.x[8,start_ind:-window],
           c=FTLEs[7,:],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a8.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a8.set_title('FTLE 8',fontsize=24)

a9 = fig.add_subplot(3,3,9,projection='3d')
scat = a9.scatter(IC_run.x[6,start_ind:-window],IC_run.x[7,start_ind:-window],IC_run.x[8,start_ind:-window],
           c=FTLEs[8,:],s=2,cmap='RdBu_r',vmax=10,vmin=-10)
a9.grid()
cbar = plt.colorbar(scat,shrink=0.5)
a9.set_title('FTLE 9',fontsize=24)

plt.savefig('Results/{}/FTBLEs_wind{}.png'.format(coupling,window),format="png",bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,40],layout='constrained',dpi=300)
a1 = fig.add_subplot(9,1,1)
scat = a1.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[0,:20000],s=2,cmap='RdBu_r',vmax=7,vmin=-7)
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 1')

a2 = fig.add_subplot(9,1,2)
scat = a2.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[1,:20000],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a2.grid()
a2.set_xlabel('x')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 2')

a3 = fig.add_subplot(9,1,3)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[2,:20000],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 3')

a4 = fig.add_subplot(9,1,4)
scat = a4.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[3,:20000],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a4.grid()
a4.set_xlabel('x')
a4.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 4')

a5 = fig.add_subplot(9,1,5)
scat = a5.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[4,:20000],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a5.grid()
a5.set_xlabel('x')
a5.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 5')

a6 = fig.add_subplot(9,1,6)
scat = a6.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[5,:20000],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a6.grid()
a6.set_xlabel('x')
a6.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 6')

a7 = fig.add_subplot(9,1,7)
scat = a7.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[6,:20000],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a7.grid()
a7.set_xlabel('x')
a7.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 7')

a8 = fig.add_subplot(9,1,8)
scat = a8.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[7,:20000],s=2,cmap='RdBu_r',vmax=3,vmin=-3)
a8.grid()
a8.set_xlabel('x')
a8.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 8')

a9 = fig.add_subplot(9,1,9)
scat = a9.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=FTLEs[8,:20000],s=2,cmap='RdBu_r',vmax=10,vmin=-10)
a9.grid()
a9.set_xlabel('x')
a9.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('FTLE 9')

plt.show()


In [ ]:
np.mean(FTLEs,axis=1)

## CLVs

In [ ]:
M = 100
orth = 1
num_CLVs = cocycle.shape[2]-2*M
print('num CLVs', num_CLVs)
CLVs_filename = '.'.join(['PenaKalnay2004CLVs', 'M{:d}'.format(M),'orth{:d}'.format(orth),'nc'])
CLVs_file = os.path.join(CLVS_DIR, CLVs_filename)

#if os.path.exists(CLVs_file):
#    CLVs = xr.open_dataset(CLVs_file).CLVs
#else:
    # Calculate CLVs
Nk = np.arange(0,M+1,orth) 

CLVs = np.zeros((dim,dim,num_CLVs),dtype=np.float64)

start = time.time()
for ti in np.arange(0,num_CLVs):
    CLVs[:,:,ti] = calculate_CLV_numerically(dim, cocycle[:,:,ti:ti+2*M], Nk, M)
end = time.time()

comp_time = (end-start)/60

print('calculated in ' + str(np.round(comp_time,4)) + ' min')
    
    ## convert to xarray
CLVs = xr.DataArray(CLVs, coords=[np.arange(1,CLVs.shape[0]+1),np.arange(1,CLVs.shape[1]+1), np.arange(M,M+num_CLVs)],
                               dims=['coordinate','CLV', 'timestep'])
    
    ## save to netCDF file
CLVs.to_dataset(name='PenaKalnay2004CLVs').to_netcdf(CLVs_file)

In [ ]:
# Calculate alignment
align = np.array(np.zeros((dim,dim,num_CLVs),dtype=np.float64))

start = time.time()
for clvi in np.arange(0,dim):
    for clvj in np.arange(0,dim):
        for tt in np.arange(0,num_CLVs):
            align[clvi,clvj,tt] = abs(np.dot(CLVs.values[:,clvi,tt],CLVs.values[:,clvj,tt]))
            
end = time.time()
comp_time = (end-start)

print('alignment calculated in ' + str(np.round(comp_time,4)) + ' sec')

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
dim2 = 9
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1,projection='3d')
        scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=align[i,j,:],s=2,cmap='RdBu_r',vmax=1,vmin=0) 
        a1.grid()
        a1.set_title('CLV{}{}'.format(i+1,j+1),fontsize=16)

cbar = plt.colorbar(scat,shrink=1.0) 

plt.savefig('Results/{}/CLV_alignment_M{}_orth{}_orientxyz_e.png'.format(coupling,M,orth),format="png",bbox_inches='tight')
plt.show()  

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
dim2 = 9
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1,projection='3d')
        scat = a1.scatter(IC_run.x[3,start_ind+M:-M],IC_run.x[4,start_ind+M:-M],IC_run.x[5,start_ind+M:-M],
           c=align[i,j,:],s=2,cmap='RdBu_r',vmax=1,vmin=0) 
        a1.grid()
        a1.set_title('CLV{}{}'.format(i+1,j+1),fontsize=16)

cbar = plt.colorbar(scat,shrink=1.0) 
plt.savefig('Results/{}/CLV_alignment_M{}_orth{}_orientxyz_t.png'.format(coupling,M,orth),format="png",bbox_inches='tight')
plt.show()  

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
dim2 = 9
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1,projection='3d')
        scat = a1.scatter(IC_run.x[6,start_ind+M:-M],IC_run.x[7,start_ind+M:-M],IC_run.x[8,start_ind+M:-M],
           c=align[i,j,:],s=2,cmap='RdBu_r',vmax=1,vmin=0) 
        a1.grid()
        a1.set_title('CLV{}{}'.format(i+1,j+1),fontsize=16)

cbar = plt.colorbar(scat,shrink=1.0) 

plt.savefig('Results/{}/CLV_alignment_M{}_orth{}_orientXYZ.png'.format(coupling,M,orth),format="png",bbox_inches='tight')
plt.show()  

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
a1 = fig.add_subplot(6,1,1)
scat = a1.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[0,1,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a1.grid()
a1.set_xlabel('x')
a1.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&2')

a2 = fig.add_subplot(6,1,2)
scat = a2.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[0,2,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a2.grid()
a2.set_xlabel('x')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&3')

a3 = fig.add_subplot(6,1,3)
scat = a3.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[0,start_ind+8000:start_ind+12000],
           c=align[1,2,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a3.grid()
a3.set_xlabel('x')
a3.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 2&3')

a4 = fig.add_subplot(6,1,4)
scat = a4.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[2,start_ind+8000:start_ind+12000],
           c=align[0,3,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a4.grid()
a4.set_xlabel('x')
a4.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&4')

a5 = fig.add_subplot(6,1,5)
scat = a5.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[2,start_ind+8000:start_ind+12000],
           c=align[0,4,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a5.grid()
a5.set_xlabel('x')
a5.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&5')

a6 = fig.add_subplot(6,1,6)
scat = a6.scatter(IC_run.time[start_ind+8000:start_ind+12000],IC_run.x[2,start_ind+8000:start_ind+12000],
           c=align[0,5,8000:12000],s=3,cmap='RdBu_r',vmax=1,vmin=0)
a6.grid()
a6.set_xlabel('x')
a6.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('CLV 1&6')

plt.show()

In [ ]:
fig = plt.figure(figsize=[24,40],layout='constrained',dpi=300)
dim2 = 9
for i in np.arange(0,dim2):
    for j in np.arange(0,i):
        a1 = fig.add_subplot(dim2,dim2,i*dim2+j+1)
        scat = a1.scatter(IC_run.x[i,:35900],IC_run.x[j,:35900],
        c=align[i,j,:35900],s=2,cmap='RdBu_r',vmax=1,vmin=0)
        a1.grid()
        a1.set_title('{}-{} axes'.format(i+1,j+1),fontsize=24)
plt.savefig('Results/{}/alignment 2D axes view M{}_orth{}_wind{}.png'.format(coupling,M,orth,window),format="png",bbox_inches='tight')
plt.show()  

## Growth rates CLVs

In [ ]:
M_GR  = 100
orth_win = 1
Nk = np.arange(orth_win-1,M_GR,orth_win)
GRs = np.empty((CLVs.shape[1],CLVs.shape[2]))

start = time.time()

for i in np.arange(0,CLVs.shape[2]):
    C = np.array(CLVs[:,:,i])
    norm_C = linalg.norm(C,axis=0)
    GR_i = np.zeros((CLVs.shape[1],Nk.shape[0]))
    ni = 0
    for tt in np.arange(0,M_GR):
        C = np.matmul(cocycle[:,:,i+tt+M],C)
        if np.isin(tt,Nk):
            GR_i[:,ni] = (linalg.norm(C,axis=0)-norm_C) / (norm_C * dt * orth_win)
            norm_C = linalg.norm(C,axis=0)
            #C[:,1] = C[:,1] - (np.dot(C[:,0], C[:,1]) / np.dot(C[:,0], C[:,0]))*C[:,0]
            #C[:,2] = C[:,2] - (np.dot(C[:,1], C[:,2]) / np.dot(C[:,1], C[:,1]))*C[:,1]
            ni += 1
    #GRs[:,i] = (linalg.norm(C,axis=0)-norm_C)/norm_C
    #GRs[:,i] = GR_i[:,-1]
    GRs[:,i] = np.mean(GR_i,axis=1)

end = time.time()
elapsed = (end-start)/60
print("Elapsed time: {} min, M = {}".format(round(elapsed,4),M_GR)) 

In [ ]:
fig = plt.figure(figsize=[24,24],layout='constrained',dpi=300)

vlims = [[9,-9],[4,-4],[2,-2],[2,-2],[2,-2],[4,-4],[6,-6],[9,-9],[9,-9]]
dim2 = 9
for i in np.arange(0,dim2):
    a1 = fig.add_subplot(3,3,i+1,projection='3d')
    scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=GRs[i,:],s=2,cmap='RdBu_r',vmax=vlims[i][0],vmin=vlims[i][1]) 
        #   c=GRs[i,:],s=2,cmap=mycmap,vmax=vlims[i][0],vmin=vlims[i][1])
    a1.grid()
    cbar = plt.colorbar(scat,shrink=0.5)
    a1.set_title('FTLE {}'.format(i+1),fontsize=24)


#cbar = plt.colorbar(scat,shrink=0.5)
#cbar.set_label('FTLE 4-6'.format(i+1))
plt.savefig('Results/{}/FTCLEs_M{}_orth{}_M_GR{}.png'.format(coupling,M,orth,M_GR),format="png",bbox_inches='tight')
plt.show()

In [ ]:
np.mean(GRs,axis=1)

## Kaplan-Yorke Dimension

In [ ]:
#FTLE_type = 'FTBLEs'
FTLE_type = 'FTCLEs'

if FTLE_type == 'FTCLEs':
    KY_FTLEs = GRs
elif FTLE_type == 'FTBLEs':
    KY_FTLEs = FTLEs
else:
    print('error: KY_FTLEs type not defined')  
    
print('Calculating KY dimensions for {}...'.format(FTLE_type))
dimKY = np.empty((KY_FTLEs.shape[1]))
t = 0
    
start = time.time()
for tt in np.arange(0,KY_FTLEs.shape[1]):
    Lyaps_t = KY_FTLEs[:,tt]
    Lyaps_ord = np.sort(Lyaps_t)
    Lyaps_ord = Lyaps_ord[::-1]
    for ll in np.arange(1,Lyaps_ord.shape[0]):
        S = np.sum(Lyaps_ord[:ll])
        if S<0:
            i_min = ll
            break
        elif ll == dim:
            i_min = ll
    
    dimKY[t] = i_min + np.sum(Lyaps_ord[:i_min])/abs(Lyaps_ord[i_min])
    t += 1
    
end = time.time()
elapsed = end-start
print("Elapsed time: {} sec".format(round(elapsed,4),M))


In [ ]:
mycmap3 = matplotlib.colors.ListedColormap([
    [0.7, 0.6, 1., 1.      ],
    [0.7, 0.6, 1., 1.      ],
    [0.5, 0.3, 0.8, 1.     ],
    [0.5, 0.3, 0.8, 1.      ], 
    [0.3, 0., 0.6, 1.      ],
    [0.3, 0., 0.6, 1.]]) 

fig = plt.figure(figsize=[24,20],layout='constrained',dpi=300)
a1 = fig.add_subplot(1,3,1,projection='3d')

if FTLE_type == 'FTBLEs':
    scat = a1.scatter(IC_run.x[0,start_ind:-window],IC_run.x[1,start_ind:-window],IC_run.x[2,start_ind:-window],
           c=dimKY,s=2,cmap='RdBu_r',vmax=9,vmin=0)
elif FTLE_type == 'FTCLEs':
    scat = a1.scatter(IC_run.x[0,start_ind+M:-M],IC_run.x[1,start_ind+M:-M],IC_run.x[2,start_ind+M:-M],
           c=dimKY,s=2,cmap='RdBu_r',vmax=9,vmin=0)
else:
     print('error: KY_FTLEs type not defined')   
a1.grid()
cbar = plt.colorbar(scat,shrink=0.3)
cbar.set_label('dimKY')

a2 = fig.add_subplot(1,3,2,projection='3d')

if FTLE_type == 'FTBLEs':
    scat = a2.scatter(IC_run.x[3,start_ind:-window],IC_run.x[4,start_ind:-window],IC_run.x[5,start_ind:-window],
           c=dimKY,s=2,cmap='RdBu_r',vmax=9,vmin=0)
elif FTLE_type == 'FTCLEs':
    scat = a2.scatter(IC_run.x[3,start_ind+M:-M],IC_run.x[4,start_ind+M:-M],IC_run.x[5,start_ind+M:-M],
           c=dimKY,s=2,cmap='RdBu_r',vmax=9,vmin=0)
else:
     print('error: KY_FTLEs type not defined')   
a1.grid()
cbar = plt.colorbar(scat,shrink=0.3)
cbar.set_label('dimKY',fontsize=16)

a3 = fig.add_subplot(1,3,3,projection='3d')

if FTLE_type == 'FTBLEs':
    scat = a3.scatter(IC_run.x[6,start_ind:-window],IC_run.x[7,start_ind:-window],IC_run.x[8,start_ind:-window],
           c=dimKY,s=2,cmap='RdBu_r',vmax=9,vmin=0)
elif FTLE_type == 'FTCLEs':
    scat = a3.scatter(IC_run.x[6,start_ind+M:-M],IC_run.x[7,start_ind+M:-M],IC_run.x[8,start_ind+M:-M],
           c=dimKY,s=2,cmap='RdBu_r',vmax=9,vmin=0)
else:
     print('error: KY_FTLEs type not defined')   
a1.grid()
cbar = plt.colorbar(scat,shrink=0.3)
cbar.set_label('dimKY',fontsize=16)

if FTLE_type == 'FTBLEs':
    print('save command commented') 
    plt.savefig('Results/{}/dimKY_FTBLEs_wind{}.png'.format(coupling,window),format="png",bbox_inches='tight')
elif FTLE_type == 'FTCLEs':
    print('save command commented') 
    plt.savefig('Results/{}/dimKY_FTCLEs_M{}_M_GR{}_orth{}.png'.format(coupling,M,M_GR,orth_win),format="png",bbox_inches='tight')
else:
     print('error: KY_FTLEs type not defined')  
plt.show()

In [ ]:
fig = plt.figure(figsize=[24,30],layout='constrained',dpi=300)
a1 = fig.add_subplot(9,1,1)
scat = a1.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[0,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a1.grid()
a1.set_xlabel('time')
a1.set_ylabel('x')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a2 = fig.add_subplot(9,1,2)
scat = a2.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[1,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a2.grid()
a2.set_xlabel('time')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a3 = fig.add_subplot(9,1,3)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[2,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a3.grid()
a3.set_xlabel('time')
a3.set_ylabel('z')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a4 = fig.add_subplot(9,1,4)
scat = a4.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[3,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a4.grid()
a4.set_xlabel('time')
a4.set_ylabel('x')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a5 = fig.add_subplot(9,1,5)
scat = a5.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[4,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a5.grid()
a5.set_xlabel('time')
a5.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a6 = fig.add_subplot(9,1,6)
scat = a6.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[5,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a6.grid()
a6.set_xlabel('time')
a6.set_ylabel('z')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a7 = fig.add_subplot(9,1,7)
scat = a7.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[6,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a7.grid()
a7.set_xlabel('time')
a7.set_ylabel('x')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a8 = fig.add_subplot(9,1,8)
scat = a8.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[7,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a8.grid()
a8.set_xlabel('time')
a8.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a9 = fig.add_subplot(9,1,9)
scat = a9.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[8,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a9.grid()
a9.set_xlabel('time')
a9.set_ylabel('z')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

plt.show()

In [ ]:
fig = plt.figure(figsize=[24,18])
a1 = fig.add_subplot(3,1,1)
scat = a1.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[3,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a1.grid()
a1.set_xlabel('time')
a1.set_ylabel('x')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a2 = fig.add_subplot(3,1,2)
scat = a2.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[4,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a2.grid()
a2.set_xlabel('time')
a2.set_ylabel('y')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')

a3 = fig.add_subplot(3,1,3)
scat = a3.scatter(IC_run.time[start_ind:start_ind+20000],IC_run.x[5,start_ind:start_ind+20000],
           c=dimKY[:20000],s=2,cmap='RdBu_r',vmax=9,vmin=0)
a3.grid()
a3.set_xlabel('time')
a3.set_ylabel('z')
cbar = plt.colorbar(scat,shrink=0.75)
cbar.set_label('dimKY')
plt.show()

In [ ]:
#25/0.